###快速排序

In [1]:
%%time
def quicksort(array):
    if len(array) < 2:
        return array
    else:
        pivot = array[0]
        less = [i for i in array[1:] if i <= pivot]
        greater = [i for i in array[1:] if i > pivot]
        return quicksort(less) + [pivot] + quicksort(greater)

a = [10,6,0,3,2,22,44,4,7,17,23]

print quicksort(a)

[0, 2, 3, 4, 6, 7, 10, 17, 22, 23, 44]
Wall time: 1e+03 µs


###广度优先搜索

使用字典生成图,使用deque生成队列，搜索第一个点，把和第一个点连接的加入队列，如果没找到就再把相连的节点加入队列末尾，同时标记已经搜索的人

In [1]:
graph={}
graph['you']=['alice','bob','claire']
graph['bob']=['you']
graph['alice']=['you']
graph['claire']=['you','mom']
graph['mom']=['claire']

In [2]:
from collections import deque

In [3]:
def person_is_seller(name):
    return name[-1] == 'm'

In [4]:
def search(name):
    search_queue = deque()
    search_queue += graph[name]
    searched = []
    while search_queue:
        person = search_queue.popleft()
        if not person in searched:
            if person_is_seller(person):
                print person + ' is a mango seller!'
                return True
            else:
                search_queue += graph[person]
                searched.append(person)
    return False

In [5]:
search('you')

mom is a mango seller!


True

###狄克斯特拉算法

不能用于有负权边的图，有的话要使用贝尔曼·福德算法（ Bellman-Ford algorithm）

使用字典（散列表）表征图

In [1]:
graph={}
graph['you']=['alice','bob','claire']
graph['start']={}
graph['start']['a']=6
graph['start']['b']=2
graph['a']={}
graph['a']['fin']=1
graph['b']={}
graph['b']['a']=3
graph['b']['fin']=5
graph['fin']={}

初始的存储开销的散列表

In [2]:
infinity=float('inf')
costs={}
costs['a']=6
costs['b']=2
costs['fin']=infinity

记录父节点的散列表

In [3]:
parents={}
parents['a']='start'
parents['b']='start'
parents['fin']=None

In [4]:
#记录处理过的节点的数组
processed=[]

找出开销最小的节点

In [14]:
def find_lowest_cost_node(costs):
    lowest_cost = float('inf')
    lowest_cost_node = None
    for node in costs:
        cost = costs[node]
        if cost < lowest_cost and node not in processed:
            lowest_cost = cost
            lowest_cost_node = node
    return lowest_cost_node

找到开销最小的点，然后看它往其他节点走的开销是不是比当前的小，小的话就更新去的节点的开销，并记录当前节点为父节点

In [15]:
node = find_lowest_cost_node(costs)
while node is not None:
    cost = costs[node]
    neighbors = graph[node]
    for n in neighbors.keys():
        new_cost = cost + neighbors[n]
        if costs[n] > new_cost:
            costs[n] = new_cost
            parents[n] = node
    processed.append(node)
    node = find_lowest_cost_node(costs)
print costs['fin']

6


###贪心算法

每次找局部最优解，最终不一定是全局最优的，但是近似的，可以用来解决NP完全问题，比如旅行商问题、集合覆盖问题等的近似解

In [2]:
# 所有要覆盖的州
states_needed = set(["mt", "wa", "or", "id", "nv", "ut","ca", "az"])
# 广播的清单
stations = {}
stations["kone"] = set(["id", "nv", "ut"])
stations["ktwo"] = set(["wa", "id", "mt"])
stations["kthree"] = set(["or", "nv", "ca"])
stations["kfour"] = set(["nv", "ut"])
stations["kfive"] = set(["ca", "az"])
# 最终选择的广播台
final_stations = set()

In [3]:
# dict.items?
print stations.items()

[('kfive', set(['ca', 'az'])), ('ktwo', set(['mt', 'id', 'wa'])), ('kthree', set(['ca', 'or', 'nv'])), ('kone', set(['ut', 'id', 'nv'])), ('kfour', set(['ut', 'nv']))]


In [4]:
# 每次都找出当前覆盖的最多的
while states_needed:
    best_station = None
    states_covered = set()
    for station, states_for_station in stations.items():
        covered = states_needed & states_for_station
        if len(covered) > len(states_covered):
            best_station = station
            states_covered = covered
            #print best_station
    final_stations.add(best_station)    
    states_needed -= states_covered

kfive
ktwo
kfive
kthree
kfive
kone


In [28]:
print final_stations

set(['ktwo', 'kthree', 'kone', 'kfive'])


###动态规划
* 单元格中的值是什么？
* 如何将这个问题划分为子问题？
* 网格的坐标轴是什么？

背包问题

把背包化成小份（根据商品的体积），看每次装完之后的背包的最优结果是什么
* 每行表示一个商品，每一列是背包体积，行的顺序无关
* 要按照行填充，按列排会和按行排有所不同
* 装的物品不能可分割，否则使用贪心算法
* 不能是相互依赖的，比如装了一个之后另一个的花费会发生变化
* 不要求背包最后恰好满（如果要求就把第二位之后的初始值设置为-inf）

In [10]:
# import numpy as np
def solve2(vlist,wlist,totalWeight,totalLength):
#     resArr = np.zeros((totalWeight)+1,dtype=np.int32)
    resArr = [0]*(totalWeight+1)
    for i in range(1,totalLength+1):
#       倒序计算从而只用一维向量即可，因为可以保证不会因为前面的改变影响后面
        for j in range(totalWeight,0,-1):
            if wlist[i] <= j:
                resArr[j] = max(resArr[j],resArr[j-wlist[i]]+vlist[i])
    return resArr[-1]

if __name__ == '__main__':
    v = [0,60,100,120]
    w = [0,10,20,30]
    weight = 50
    n = 3
    result = solve2(v,w,weight,n)
    print(result)

220


其它的一些问题的解决也用到了动态规划
* 判断和当前的词符相似的词（编辑距离），使用最长公共子串（连续的）或者最长公共子序列（不一定连续）
* DNA序列相似性
* git中判断文件的修改情况
* word判断断行的位置

###k最邻近算法

归类位离得近的类